In [ ]:
import json

with open('gcp_iap_conf.json') as f:
    kfp_conf = json.load(f)

kfp_host = kfp_conf["host"]
kfp_client_id_web = kfp_conf["client_id_web"]
kfp_namespace = kfp_conf["namespace"]
kfp_client_id_desktop = kfp_conf["client_id_desktop"]
kfp_client_secret_desktop = kfp_conf["client_secret_desktop"]

In [ ]:
import google.auth
from google_auth_oauthlib.flow import InstalledAppFlow
import requests

In [ ]:
CLIENT_ID = kfp_client_id_desktop
CLIENT_SECRET = kfp_client_secret_desktop

SCOPES = ['https://www.googleapis.com/auth/userinfo.email']

REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

In [ ]:
flow = InstalledAppFlow.from_client_secrets_file(
    'client_id_desktop_secret.json',
    scopes=SCOPES,
    redirect_uri=REDIRECT_URI
)

authorization_url, _ = flow.authorization_url()

In [ ]:
print(f'Open the following URL in your web browser:\n{authorization_url}')
auth_code = input('Enter the authorization code here: ')

In [ ]:
token_url = 'https://oauth2.googleapis.com/token'
token_data = {
    'code': auth_code,
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'redirect_uri': REDIRECT_URI,
    'grant_type': 'authorization_code'
}

In [ ]:
token_response = requests.post(token_url, data=token_data)

if token_response.status_code == 200:
    token_info = token_response.json()
    id_token = token_info.get('id_token')
    refresh_token = token_info.get('refresh_token')
    #print(f'ID Token: {id_token}')
else:
    print(f'Error status code: {token_response.status_code}')
    print(token_response.text)

In [ ]:
headers = {
    'Authorization': f'Bearer {id_token}',
    'Content_type': 'application/json'
}

oauth_response = requests.get(kfp_host, headers=headers)

if oauth_response.status_code == 200:
    print(f'Success!\n Response: {oauth_response.text}')
else:
    print(f'Error status code: {oauth_response.status_code}')
    print(oauth_response.text)

In [ ]:
# In case if you want to refresh the token

refresh_token_url = 'https://oauth2.googleapis.com/token'
refresh_token_data = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'refresh_token': refresh_token,
    'grant_type': 'refresh_token'
}

refresh_token_response = requests.post(token_url, data=token_data)

if refresh_token_response.status_code == 200:
        token_info = refresh_token_response.json()
        refreshed_access_token = token_info.get('access_token')
        # print(f'Refreshed Access Token: {refreshed_access_token}')
else:
    print(f'Error Status code: {refresh_token_response.status_code}')
    print(refresh_token_response.text)